In [ ]:
!pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 54.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21708 sha256=03dc634ad73778bdbcd3b2ab583f76cb875bc2fc03d97429b03f4aebc09d374d
  Stored in directory: /root/.cache/pip/wheels/ba/9f/70/7795228568b81b57a8932755938da9fb1f291b0576752604aa
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=e8639b8d105f00d935031814c76c266ab17c6f85fc68e8eb0dd43496f1bbc181
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c9

In [ ]:
!pip install transformers
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr
import transformers
import spacy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

# Load spacy model
nlp = spacy.load("en_core_web_sm")

# Load transformers models for abstractive summarization
models = {
    "GPT-3": "EleutherAI/gpt-neo-2.7B",
    "BART": "facebook/bart-large-cnn",
    "T5": "t5-large"
}

# Load sumy models for extractive summarization
sumy_models = {
    "TextRank": TextRankSummarizer,
    "LSA": LsaSummarizer,
    "LUHN": LuhnSummarizer
}

def count_words(text):
    doc = nlp(text)
    return len(doc)

def summarize_text(text, summarizer_type, model=None, sumy_model=None):
    if summarizer_type == "Abstractive":
        # Load the selected model
        tokenizer = transformers.AutoTokenizer.from_pretrained(models[model])
        model = transformers.AutoModelWithLMHead.from_pretrained(models[model])

        # Summarize the text using the selected model
        inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
        outputs = model.generate(inputs, max_length=200, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    elif summarizer_type == "Extractive":
        # Summarize the text using the selected sumy model
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        summarizer = sumy_models[sumy_model]()
        summary = ""
        for sentence in summarizer(parser.document, 2):
            summary += str(sentence) + " "

    return summary

# Define the Gradio interface
input_text = gr.inputs.Textbox(lines=15, label="Enter text to summarize")
summarizer_type = gr.inputs.Radio(["Abstractive", "Extractive"], label="Select the type of summarizer")
abstractive_model = gr.inputs.Radio(list(models.keys()), label="Select the abstractive model", default=None)
extractive_model = gr.inputs.Radio(list(sumy_models.keys()), label="Select the extractive model", default=None)
output_text = gr.outputs.Textbox(label="Summary")
input_word_count = gr.outputs.Textbox(label="Input Word Count")
output_word_count = gr.outputs.Textbox(label="Output Word Count")

def summarizer_callback(text, summarizer_type, abstractive_model=None, extractive_model=None):
    input_word_count = count_words(text)
    if summarizer_type == "Abstractive":
        summary = summarize_text(text, summarizer_type, model=abstractive_model)
    else:
        summary = summarize_text(text, summarizer_type, sumy_model=extractive_model)
    output_word_count = count_words(summary)
    return summary, str(input_word_count), str(output_word_count)

summarizer_interface = gr.Interface(summarizer_callback, 
                                    inputs=[input_text, summarizer_type, abstractive_model, extractive_model], 
                                    outputs=[output_text, input_word_count, output_word_count],
                                    title="Text Summarization",
                                    description="Summarize text using abstractive or extractive summarization",
                                    theme="default")

summarizer_interface.launch() # Launch the interface


/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:183: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>